# Load the overlay

In [6]:
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/User_overlay/Audio_test.bit')

i2s_RX=overlay.Audio_controller.i2s_audio_stream.i2s_receiver_0
i2s_TX=overlay.Audio_controller.i2s_audio_stream.i2s_transmitter_0
MUX_sel=overlay.Audio_controller.mux_sel

/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:691: UserWarning: Interrupt irq not created: Could not find UIO device for interrupt pin for IRQ number 62
  interrupt, str(e)))
/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:691: UserWarning: Interrupt irq not created: Could not find UIO device for interrupt pin for IRQ number 63
  interrupt, str(e)))


# Read test
Axi protocol for I2S RX and TX
* Reg 0x00 --> Version
* Reg 0x04 --> Configuration (Read only)
* Reg 0x08 --> Enable(0x09)/Disable(0x08)

Go to Logic Core resurces from Xilinx for more info

In [4]:
i2s_RX.read(0x08)
hex(value)

'0x8'

In [5]:
i2s_TX.read(0x20)
hex(value)

'0x8'

# Lybrary import

In [10]:
import pynq
from pynq import GPIO
import pynq.lib
import pynq.lib.video
import pynq.lib.audio
import cffi
from pynq import PL
from pynq import GPIO
from pynq.uio import get_uio_index

In [11]:
#define the audio codec
pAudio=overlay.Audio_controller.audio_codec_ctrl_0

In [35]:
#Configure the audio codec
pAudio.configure(sample_rate=96000)
pAudio.set_volume(35)
pAudio.select_line_in()

In [39]:
#test
pAudio.bypass(10)

# I2S module functions

In [17]:
def config_TX_i2s(sample_rate=48000, En=True):
    if sample_rate == 48000:
        i2s_TX.write(0x20,0x08)
    elif sample_rate == 96000:
        i2s_TX.write(0x20,0x04)
    else:
        i2s_TX.write(0x20,0x08)
    if En:
        i2s_TX.write(0x08,0x09)
    else:
        i2s_TX.write(0x08,0x08)

In [18]:
def config_RX_i2s(sample_rate=48000, En=True):
    if sample_rate == 48000:
        i2s_RX.write(0x20,0x08)
    elif sample_rate == 96000:
        i2s_RX.write(0x20,0x04)
    else:
        i2s_RX.write(0x20,0x08)
    for i in range(6):
        i2s_RX.write(0x50+i*4,0x00)
    if En:
        i2s_RX.write(0x08,0x09)
    else:
        i2s_RX.write(0x08,0x08)

# Enable continuous bypass from audio codec

In [19]:
def continuous_bypass(En=True, volume=30):
    if volume > 63:
        volume=63
    if volume < 0:
        volume=0
    if En:
        #overlay.mux_sel.write(0,0)
        pAudio.write_audio_REG(0x1D,0x00)
        pAudio.write_audio_REG(0x1F,0x00)
        pAudio.write_audio_REG(0x1C,0x21)
        pAudio.write_audio_REG(0x1E,0x41)
        reg_vol=volume<< 2 |0x3
        pAudio.write_audio_REG(0x23,reg_vol)
        pAudio.write_audio_REG(0x24,reg_vol)
        #overlay.mux_sel.write(0,1)
    else:
        pAudio.write_audio_REG(0x1D,0x00)
        pAudio.write_audio_REG(0x1F,0x00)
        pAudio.write_audio_REG(0x1C,0x00)
        pAudio.write_audio_REG(0x1E,0x00)
        pAudio.write_audio_REG(0x23,0xE5)
        pAudio.write_audio_REG(0x24,0xE5)
        #overlay.mux_sel.write(0,0)

In [256]:
continuous_bypass(False,10)

In [48]:
continuous_bypass(True,45)

In [40]:
overlay.Audio_controller.mux_sel.write(0x0,0x1)

In [34]:
config_RX_i2s(96000,False)
config_TX_i2s(96000,False)

In [30]:
config_RX_i2s(48000,True)
config_TX_i2s(48000,True)

In [41]:
config_RX_i2s(96000,True)
config_TX_i2s(96000,True)

In [47]:
pAudio.write_audio_REG(0x14,0x00)

In [46]:
reg_val=0
for i in range(6):
    value=i2s_RX.read(0x50+i*4)
    reg_val=reg_val << 16 | value
print(hex(reg_val))
reg_val=0
for i in range(6):
    value=i2s_TX.read(0x50+i*4)
    reg_val=reg_val << 16 | value
print(hex(reg_val))

0x0
0x0
